In [5]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt


In [27]:
# Load the cropped image
image = Image.open('/Users/mikaildemir/Desktop/Image-Augmentation-Salmonella/Array.tif')

# Define the new patch size based on the updated image dimensions
patch_size = (55, 53)  # Width x Height

# Convert the image to a numpy array for easy manipulation
image_array = np.array(image)

# Define the number of rows and columns in the grid
rows = 7  # 7 rows
cols = 10  # 10 columns

# Initialize a list to store all patches
patches = []

# Loop through each row and column to extract all patches
for row in range(rows):
    for col in range(cols):
        # Calculate the coordinates for the top-left corner of each patch
        x = col * patch_size[0]
        y = row * patch_size[1]
        
        # Extract the patch
        patch = image_array[y:y + patch_size[1], x:x + patch_size[0]]
        patches.append(patch)
patches = patches[:61] + [patches[-1]]
# The list `patches` now contains all extracted patches from the image
print(f"Total patches extracted: {len(patches)}")


Total patches extracted: 62


In [112]:
# Your list of concentration levels (62 values)
concentration_levels = [
    0, 0, 20, 20, 0, 0, 20, 10, 0, 10, 10, 0, 50, 0, 0, 50, 0, 50, 20, 0, 
    0, 0, 0, 20, 0, 10, 10, 50, 0, 0, 20, 50, 10, 0, 10, 0, 20, 0, 0, 20, 
    0, 0, 10, 0, 0, 50, 50, 0, 10, 50, 50, 0, 50, 20, 0, 10, 0, 0, 20, 0, 
    0, 0
]

# Make sure the number of patches matches the number of concentration levels
assert len(patches) == len(concentration_levels), "Mismatch between patches and concentration levels."

# Create a list of tuples where each tuple contains a patch and its corresponding concentration level
labeled_patches = [(patch, concentration_levels[i]) for i, patch in enumerate(patches)]



In [111]:
import albumentations as A
import cv2
import os

# Define aggressive RGB Shift and Channel Shuffle augmentations
rgb_shift_augmentation = A.Compose([
    A.RGBShift(r_shift_limit=100, g_shift_limit=100, b_shift_limit=100, p=1.0)
])

channel_shuffle_augmentation = A.Compose([
    A.ChannelShuffle(p=1.0)
])

# Create directories if they don’t exist
os.makedirs("original_patches", exist_ok=True)
os.makedirs("augmented_images", exist_ok=True)

# List to store augmented images along with their labels
augmented_labeled_patches = []

# Loop through all patches and generate augmented versions
for idx, (patch, label) in enumerate(labeled_patches):
    # Save original patch
    original_path = f"original_patches/patch_{idx}.png"
    cv2.imwrite(original_path, cv2.cvtColor(patch, cv2.COLOR_RGB2BGR))
    
    # Generate and save 4 augmented images for each patch
    for aug_idx in range(1, 5):
        if aug_idx <= 2:
            # Apply RGB Shift for the first 2 augmentations
            augmented_image = rgb_shift_augmentation(image=patch)['image']
        else:
            # Apply Channel Shuffle for the last 2 augmentations
            augmented_image = channel_shuffle_augmentation(image=patch)['image']
        
        # Save the augmented image
        augmented_path = f"augmented_images/augmented_{idx}_{aug_idx}.png"
        cv2.imwrite(augmented_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
        
        # Append the augmented image and label to augmented_labeled_patches
        augmented_labeled_patches.append((augmented_image, label))

print("All images saved successfully.")

# Now, `augmented_labeled_patches` contains all original and augmented images with labels


All images saved successfully.
